# Fraud Detection System
## Combined Location, Behavioral, and ML Analysis

## 1. Installation & Imports

In [27]:
import datetime
import requests
import numpy as np
import pandas as pd
from geopy.distance import geodesic
from sklearn.ensemble import IsolationForest
from scipy.stats import zscore, poisson
from collections import defaultdict, Counter
import matplotlib.pyplot as plt
from faker import Faker
import hashlib
import json
import csv
import os
from requests.adapters import HTTPAdapter
from urllib3.util.retry import Retry

%matplotlib inline

## 2. Configuration

In [28]:
class Config:
    # File paths
    TRANSACTION_HISTORY = "transactions.csv"
    FRAUD_CASES = "fraud_cases.json"
    ML_MODEL_PATH = "isolation_forest_model.pkl"
    
    # APIs
    IP_GEOLOCATION_API = "http://ip-api.com/json/"
    VPN_API_URL = "https://ipqualityscore.com/api/json/ip//aDPrnvyLMFpEN07h9XJdMNZvnodxe0ly"
    
    # Thresholds
    IMPOSSIBLE_TRAVEL_KPH = 800
    AMOUNT_Z_SCORE = 3.5
    BENFORD_PROBS = [0.301, 0.176, 0.125, 0.097, 0.079, 0.067, 0.058, 0.051, 0.046]
    TIME_OF_DAY_RISK = {h: 0.8 if h in [0,1,2,3] else 0.1 for h in range(24)}
    
    # Lists
    BLACKLISTED_COUNTRIES = {"North Korea", "Iran", "Syria"}
    HIGH_RISK_MERCHANTS = {"Gambling", "Cryptocurrency", "Adult"}
    
    # ML Settings
    ML_FEATURES = ['amount', 'time_risk', 'velocity', 'entropy', 'benford_deviation']

## 3. Transaction Class with Geolocation

In [29]:
class FraudTransaction:
    _geo_cache = {}
    _session = requests.Session()
    
    # Configure retries with backoff
    retries = Retry(
        total=3,
        backoff_factor=0.3,
        status_forcelist=[500, 502, 503, 504],
        allowed_methods={'GET'}
    )
    
    _session.mount('http://', HTTPAdapter(max_retries=retries))
    _session.mount('https://', HTTPAdapter(max_retries=retries))
    def __init__(self, transaction_id, user_id, amount, timestamp, merchant, ip, device):
        self.transaction_id = transaction_id
        self.user_id = user_id
        self.amount = float(amount)
        self.timestamp = datetime.datetime.strptime(timestamp, "%Y-%m-%d %H:%M:%S")
        self.merchant = merchant
        self.ip = ip
        self.device = device
        
        # Geolocation data
        self.geo = self._get_geolocation()
        self.country = self.geo.get('country', 'Unknown')
        self.lat = self.geo.get('lat', 0)
        self.lon = self.geo.get('lon', 0)
        self.timezone = self.geo.get('timezone', 'UTC')
        
        # Security checks
        self.is_proxy = self._check_vpn_proxy()
    

    def _get_geolocation(self):
        """Improved geolocation lookup with retries and caching"""
        if self.ip in self._geo_cache:
            return self._geo_cache[self.ip]
        
        try:
            # Try primary API with timeout
            response = self._session.get(
                f"{Config.IP_GEOLOCATION_API}/{self.ip}",
                timeout=(3.05, 5),  # Connect timeout 3s, read timeout 5s
                headers={'User-Agent': 'FraudDetectionSystem/1.0'}
            )
            if response.ok and response.json().get('status') == 'success':
                data = response.json()
                geo_data = {
                    'country': data.get('country', 'Unknown'),
                    'lat': data.get('lat', 0),
                    'lon': data.get('lon', 0),
                    'timezone': data.get('timezone', 'UTC')
                }
                self._geo_cache[self.ip] = geo_data
                return geo_data
            
            # Fallback to secondary API
            return self._fallback_geolookup()
            
        except Exception as e:
            print(f"Geolocation failed: {e}")
            return self._get_cached_default()

    def _fallback_geolookup(self):
        """Try backup geolocation service"""
        try:
            response = self._session.get(
                f"https://geolocation-db.com/json/{self.ip}",
                timeout=3
            )
            if response.ok:
                data = response.json()
                return {
                    'country': data.get('country_name', 'Unknown'),
                    'lat': float(data.get('latitude', 0)),
                    'lon': float(data.get('longitude', 0)),
                    'timezone': data.get('timezone', 'UTC')
                }
        except Exception as e:
            print(f"Fallback geolookup failed: {e}")
            
        return self._get_cached_default()


    def _get_cached_default(self):
        """Return cached values for common IPs"""
        common_ips = {
            '8.8.8.8': {'country': 'United States', 'lat': 37.7510, 'lon': -97.8220},
            '1.1.1.1': {'country': 'United States', 'lat': 34.0522, 'lon': -118.2437}
        }
        return common_ips.get(self.ip, {'country': 'Unknown', 'lat': 0, 'lon': 0})
    
    def _check_vpn_proxy(self):
        try:
            response = requests.get(
                f"{Config.VPN_API_URL}{self.ip}",
                params={'strictness': 1}
            )
            return response.json().get('vpn', False) or response.json().get('proxy', False)
        except:
            return False

## 4. Fraud Detector Engine

In [30]:
class AdvancedFraudDetector:
    def __init__(self):
        self.history = defaultdict(lambda: {
            'timestamps': [], 'amounts': [], 'locations': [],
            'devices': [], 'countries': [], 'timezones': []
        })
        self.fraud_cases = []
        self.ml_model = None  # Initialize as None
        self._load_history()
        self._initialize_model()
        
    def _initialize_model(self):
        """Train model if historical data exists"""
        if os.path.exists(Config.TRANSACTION_HISTORY):
            df = pd.read_csv(Config.TRANSACTION_HISTORY)
            if len(df) >= 100:  # Minimum samples for training
                features = self._create_training_features(df)
                self.ml_model = IsolationForest(contamination=0.01)
                self.ml_model.fit(features)
    
    def _create_training_features(self, df):
        """Create features from historical data"""
        features = []
        for _, row in df.iterrows():
            user_history = self.history[row['user_id']]
            fake_transaction = FraudTransaction(
                row['transaction_id'], row['user_id'], row['amount'],
                row['timestamp'], 'dummy', '0.0.0.0', 'dummy_device'
            )
            features.append(self._create_ml_features(fake_transaction, user_history))
        return np.array(features)


    def detect_fraud(self, transaction):
        reasons = []
        user_id = transaction.user_id
        user_history = self.history[user_id]
        
        # Rule-based checks
        if transaction.country in Config.BLACKLISTED_COUNTRIES:
            reasons.append(f"Blacklisted country: {transaction.country}")
        
        if transaction.merchant in Config.HIGH_RISK_MERCHANTS:
            reasons.append(f"High-risk merchant: {transaction.merchant}")
            
        if transaction.is_proxy:
            reasons.append("VPN/Proxy detected")
        
        # Location analysis
        if user_history['locations']:
            last_loc = user_history['locations'][-1]
            time_diff = (transaction.timestamp - user_history['timestamps'][-1]).total_seconds() / 3600
            distance = geodesic(last_loc, (transaction.lat, transaction.lon)).km
            
            if time_diff > 0 and (distance / time_diff) > Config.IMPOSSIBLE_TRAVEL_KPH:
                reasons.append(f"Impossible travel: {distance:.1f}km in {time_diff:.1f}hrs")
        
        # Behavioral analysis
        if len(user_history['amounts']) > 5:
            z = zscore(user_history['amounts'] + [transaction.amount])[-1]
            if abs(z) > Config.AMOUNT_Z_SCORE:
                reasons.append(f"Amount anomaly (z={z:.2f})")
        
        # ML prediction only if model is trained
        if self.ml_model:
            ml_features = self._create_ml_features(transaction, user_history)
            ml_risk = self.ml_model.decision_function([ml_features])[0]
            if ml_risk < -0.5:
                reasons.append(f"ML anomaly score: {ml_risk:.2f}")
        else:
            reasons.append("ML model not initialized")
        
        # Update history and retrain periodically
        self._update_history(transaction)
        if len(self.history[user_id]['amounts']) % 100 == 0:
            self._retrain_model()
     
        # Save fraud case
        if reasons:
            # In detect_fraud() method
            self.fraud_cases.append({
                'id': int(transaction.transaction_id),  # Convert numpy int to Python int
                'user': int(transaction.user_id),
                'amount': float(transaction.amount),
                'reasons': [str(r) for r in reasons],  # Ensure all reasons are strings
                'timestamp': transaction.timestamp.isoformat(),
                'location': (float(transaction.lat), float(transaction.lon))  # Convert numpy floats
            })
            return True, reasons
        
        return False, []

    def _retrain_model(self):
        """Periodically retrain the model"""
        features = []
        targets = []
        for user_id, data in self.history.items():
            for i in range(len(data['amounts'])):
                fake_transaction = FraudTransaction(
                    i, user_id, data['amounts'][i],
                    data['timestamps'][i].strftime("%Y-%m-%d %H:%M:%S"),
                    'dummy', '0.0.0.0', 'dummy_device'
                )
                features.append(self._create_ml_features(fake_transaction, data))
                targets.append(1 if i in [c['id'] for c in self.fraud_cases] else 0)
                
        if len(features) >= 100:
            self.ml_model = IsolationForest(contamination=0.01)
            self.ml_model.fit(features)
            
                      
    def _create_ml_features(self, transaction, history):
        # Feature engineering for ML model
        features = {
            'amount': transaction.amount,
            'time_risk': Config.TIME_OF_DAY_RISK[transaction.timestamp.hour],
            'velocity': len([ts for ts in history['timestamps'] 
                          if (transaction.timestamp - ts).total_seconds() < 3600]),
            'entropy': self._calculate_entropy(history['countries']),
            'benford_deviation': self._benford_deviation(transaction.amount)
        }
        return [features[k] for k in Config.ML_FEATURES]
    
    def _calculate_entropy(self, countries):
        counts = Counter(countries)
        probs = [c/len(countries) for c in counts.values()]
        return -sum(p * np.log(p) for p in probs)

    def _benford_deviation(self, amount):
        first_digit = int(str(amount).lstrip('0')[0])
        expected = Config.BENFORD_PROBS[first_digit-1]
        
        # Count all historical amounts
        total_amounts = sum(len(user['amounts']) for user in self.history.values())
        if total_amounts == 0:
            return 0
        
        # Count matches across all users
        actual_count = sum(
            1 for user in self.history.values() 
            for amt in user['amounts'] 
            if int(str(amt).lstrip('0')[0]) == first_digit
        )
        
        actual = actual_count / total_amounts
        return abs(expected - actual)

    def _update_history(self, transaction):
        """Update transaction history CSV with proper columns"""
        new_data = pd.DataFrame({
            'transaction_id': [int(transaction.transaction_id)],
            'user_id': [int(transaction.user_id)],
            'timestamp': [transaction.timestamp.strftime("%Y-%m-%d %H:%M:%S")],
            'amount': [float(transaction.amount)],
            'lat': [float(transaction.lat)],
            'lon': [float(transaction.lon)],
            'country': [str(transaction.country)],
            'device': [str(transaction.device)],
            'merchant': [str(transaction.merchant)]
        })
        
        # Append to CSV with headers if file doesn't exist
        header = not os.path.exists(Config.TRANSACTION_HISTORY)
        new_data.to_csv(Config.TRANSACTION_HISTORY, 
                       mode='a', 
                       header=header, 
                       index=False)


    def _load_history(self):
        """Load history with proper column validation"""
        if os.path.exists(Config.TRANSACTION_HISTORY):
            df = pd.read_csv(Config.TRANSACTION_HISTORY)
            
            # Validate required columns
            required_columns = {'transaction_id', 'user_id', 'timestamp', 
                               'amount', 'lat', 'lon', 'country'}
            if not required_columns.issubset(df.columns):
                raise ValueError("Transaction history file missing required columns")
                
            for _, row in df.iterrows():
                user_id = row['user_id']
                # Convert pandas nulls to None
                ts = datetime.datetime.strptime(row['timestamp'], "%Y-%m-%d %H:%M:%S") 
                self.history[user_id]['timestamps'].append(ts)
                self.history[user_id]['amounts'].append(float(row['amount']))
                self.history[user_id]['locations'].append((float(row['lat']), float(row['lon'])))
                self.history[user_id]['countries'].append(str(row['country']))

                
    def visualize_fraud_patterns(self):
        df = pd.read_csv(Config.TRANSACTION_HISTORY)
        df['timestamp'] = pd.to_datetime(df['timestamp'])
        
        plt.figure(figsize=(15,5))
        plt.subplot(1,2,1)
        df.groupby(df['timestamp'].dt.hour)['amount'].mean().plot(title='Transaction Amount by Hour')
        
        plt.subplot(1,2,2)
        pd.Series([len(c['reasons']) for c in self.fraud_cases]).value_counts().plot.pie(title='Fraud Reasons Count')
        plt.show()

    # Save fraud cases
    def save_fraud_cases(self):
        """Convert numpy types to Python native types before saving"""
        def convert_types(obj):
            if isinstance(obj, np.generic):
                return obj.item()
            elif isinstance(obj, datetime):
                return obj.isoformat()
            return obj
    
        serializable_cases = []
        for case in self.fraud_cases:
            converted = {k: convert_types(v) for k, v in case.items()}
            converted['reasons'] = [str(r) for r in converted['reasons']]  # Convert any non-string reasons
            serializable_cases.append(converted)
    
        with open(Config.FRAUD_CASES, 'w') as f:
            json.dump(serializable_cases, f, default=convert_types, indent=2)
    

## 5. Generate Test Data

In [31]:
def generate_test_transactions(num=50):
    fake = Faker()
    transactions = []
    
    for _ in range(num):
        # Generate 90% normal transactions
        if np.random.rand() < 0.9:
            transactions.append(FraudTransaction(
                transaction_id=np.random.randint(1000,9999),
                user_id=np.random.choice([101, 102, 103]),
                amount=np.random.uniform(10, 500),
                timestamp=fake.date_time_this_year().strftime("%Y-%m-%d %H:%M:%S"),
                merchant=np.random.choice(["Retail", "Food", "Services"]),
                ip=fake.ipv4(),
                device=f"DEV_{np.random.randint(1000,9999)}"
            ))
        # 10% fraudulent transactions
        else:
            transactions.append(FraudTransaction(
                transaction_id=np.random.randint(1000,9999),
                user_id=np.random.choice([101, 102, 103]),
                amount=np.random.uniform(5000, 10000),
                timestamp=fake.date_time_this_year().strftime("%Y-%m-%d %H:%M:%S"),
                merchant=np.random.choice(["Gambling", "Cryptocurrency"]),
                ip="185.194.36.1",  # Known risky IP
                device=f"MAL_{np.random.randint(1000,9999)}"
            ))
    return transactions

## 6. Run Detection System

In [ ]:
# Initialize system
fraud_system = AdvancedFraudDetector()

# First 100 transactions (model not ready)
initial_transactions = generate_test_transactions(50)
for txn in initial_transactions:
    fraud_system.detect_fraud(txn)

# Subsequent transactions (model will be trained)
fraud_system._retrain_model()
test_transactions = generate_test_transactions(100)
results = []
for txn in test_transactions:
    is_fraud, reasons = fraud_system.detect_fraud(txn)
    results.append({
        'id': txn.transaction_id,
        'amount': txn.amount,
        'fraud': is_fraud,
        'reasons': reasons
    })

# Show results
results_df = pd.DataFrame(results)
print("Detection Results:")
print(results_df.head())

# Visualization
fraud_system.visualize_fraud_patterns()

fraud_system.save_fraud_cases()


Fallback geolookup failed: could not convert string to float: 'Not found'
Fallback geolookup failed: could not convert string to float: 'Not found'
Fallback geolookup failed: could not convert string to float: 'Not found'
Fallback geolookup failed: could not convert string to float: 'Not found'
Fallback geolookup failed: could not convert string to float: 'Not found'
Fallback geolookup failed: could not convert string to float: 'Not found'
Fallback geolookup failed: could not convert string to float: 'Not found'
Fallback geolookup failed: could not convert string to float: 'Not found'
Fallback geolookup failed: could not convert string to float: 'Not found'
Fallback geolookup failed: could not convert string to float: 'Not found'
Fallback geolookup failed: could not convert string to float: 'Not found'
Fallback geolookup failed: could not convert string to float: 'Not found'
Fallback geolookup failed: could not convert string to float: 'Not found'
Fallback geolookup failed: could not c